In [8]:

import os

os.environ["KERAS_BACKEND"] = "tensorflow"
from preprocessing import getdataset,character_to_number, number_to_character

MIN_FREQ = 10

dataset , table, MAX_LEN = getdataset()
table.crop(MIN_FREQ)
print(table.vocab_size)


shocolat
11261


In [9]:
from network import make_model

EMBED_DIM = 128
NUM_HEADS = 3
NUM_BLOCS = 8
hidden_dim = 512
model = make_model(MAX_LEN, table.vocab_size, EMBED_DIM, NUM_HEADS,NUM_BLOCS,hidden_dim)

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 62)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_1  │ (None, 62, 128)        │     1,449,344 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_8             │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_9             │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_10            │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_11            │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_12            │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_13            │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_14            │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_15            │ (None, 62, 128)        │       214,912 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 62, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 62, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 62, 11261)      │       731,965 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,908,861 (14.91 MB)

 Trainable params: 3,908,861 (14.91 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])


In [12]:
X = tf.convert_to_tensor(dataset)
Y = np.zeros(X.shape).astype(int)
Y[:,0:-1] = np.array(X[:,1:])
Y = tf.convert_to_tensor(Y)
print(X)
print(Y)
#Y = tf.one_hot(Y, depth=table.vocab_size)
# change X to one_hot_vectors

print("pourcentage sur la prédiction de que des 0 : ",np.sum(X == 0)/Y.shape[0]/Y.shape[1]*100)

tf.Tensor(
[[    2     3     4 ...     0     0     0]
 [    2    10    11 ...     0     0     0]
 [    2    10    11 ...     0     0     0]
 ...
 [    2 78691     1 ...     0     0     0]
 [    2 78692     1 ...     0     0     0]
 [    2 78693     1 ...     0     0     0]], shape=(43352, 62), dtype=int32)
tf.Tensor(
[[    3     4     5 ...     0     0     0]
 [   10    11    26 ...     0     0     0]
 [   10    11    41 ...     0     0     0]
 ...
 [78691     1     0 ...     0     0     0]
 [78692     1     0 ...     0     0     0]
 [78693     1     0 ...     0     0     0]], shape=(43352, 62), dtype=int64)
pourcentage sur la prédiction de que des 0 :  63.60669448594849


In [13]:
# example : 
tweet = np.array(X[0])

print("tweet : ",[number_to_character(x,table) for x in tweet])

tweet :  ['<sos>', 'be', 'sure', 'to', 'tune', 'in', 'and', 'watch', 'donald', 'trump', 'on', 'late', 'night', 'with', 'david', 'letterman', 'as', 'he', 'presents', 'the', 'top', 'ten', 'list', 'tonight!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
history = model.fit(X, Y, epochs=30, batch_size=64,verbose=1)
model.save("model.h5")

Epoch 1/30


W0000 00:00:1735305958.303419   30594 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-12-27 14:26:00.301623: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 68 bytes spill stores, 72 bytes spill loads

2024-12-27 14:26:00.320512: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_58', 1744 bytes spill stores, 1212 bytes spill loads

2024-12-27 14:26:00.359610: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 196 bytes spill stores, 196 bytes spill loads

2024-12-27 14:26:00.472189: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptx

  2/678 ━━━━━━━━━━━━━━━━━━━━ 34s 51ms/step - loss: nan - sparse_categorical_accuracy: 0.1641       

2024-12-27 14:26:08.274749: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'copy_fusion_1', 608 bytes spill stores, 608 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 52 bytes spill stores, 52 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'copy_fusion', 608 bytes spill stores, 608 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 52 bytes spill stores, 52 bytes spill loads



676/678 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: nan - sparse_categorical_accuracy: 0.6466

W0000 00:00:1735305996.808437   30594 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-12-27 14:26:38.631954: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 32 bytes spill stores, 32 bytes spill loads

2024-12-27 14:26:38.648345: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 208 bytes spill stores, 204 bytes spill loads

2024-12-27 14:26:38.740911: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_58', 1744 bytes spill stores, 1212 bytes spill loads

2024-12-27 14:26:38.758060: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptx

678/678 ━━━━━━━━━━━━━━━━━━━━ 58s 58ms/step - loss: nan - sparse_categorical_accuracy: 0.6466
Epoch 2/30
  1/678 ━━━━━━━━━━━━━━━━━━━━ 43s 65ms/step - loss: nan - sparse_categorical_accuracy: 0.6318

2024-12-27 14:26:47.435423: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_14', 384 bytes spill stores, 384 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_13', 388 bytes spill stores, 388 bytes spill loads



678/678 ━━━━━━━━━━━━━━━━━━━━ 27s 40ms/step - loss: nan - sparse_categorical_accuracy: 0.6516
Epoch 3/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 27s 40ms/step - loss: nan - sparse_categorical_accuracy: 0.6512
Epoch 4/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 28s 42ms/step - loss: nan - sparse_categorical_accuracy: 0.6526
Epoch 5/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 27s 40ms/step - loss: nan - sparse_categorical_accuracy: 0.6525
Epoch 6/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 27s 40ms/step - loss: nan - sparse_categorical_accuracy: 0.6532
Epoch 7/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 27s 39ms/step - loss: nan - sparse_categorical_accuracy: 0.6526
Epoch 8/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 28s 42ms/step - loss: nan - sparse_categorical_accuracy: 0.6519
Epoch 9/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 25s 37ms/step - loss: nan - sparse_categorical_accuracy: 0.6523
Epoch 10/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - loss: nan - sparse_categorical_accuracy: 0.6514
Epoch 11/30
678/678 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - loss: nan - spars

In [ ]:
history = model.fit(X, Y, epochs=30, batch_size=128,verbose=1)
model.save("model_2.h5")

In [ ]:
model.save("model_3.h5")

In [9]:
text_to_begin_with = "the".split(" ")
begining = [ table.table[i] for i in text_to_begin_with ]
nb_to_repeat = MAX_LEN - len(begining) -1
input = np.array([[2] + begining + [0] * nb_to_repeat])
exit = False
nb_iter = len(text_to_begin_with)+1
max_iter = MAX_LEN

print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

while not exit:
    output = model.predict(input)

    input[0,nb_iter] = np.argmax(output[0,nb_iter-1])
    nb_iter += 1
    if nb_iter == max_iter:
        exit = True
    
print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

<sos>the<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


2024-12-27 14:16:14.528836: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17', 88 bytes spill stores, 88 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━